In [134]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request, response, parse
import re
import pickle
path_file = '/Users/gowitheflow/Documents/ETALAB/algo_gouv/'

In [17]:
import random
import time
from IPython.display import clear_output

In [150]:
resultObj = open('RESULT_DICT_google','rb')
RESULT_DICT = pickle.load(resultObj)

In [151]:
RESULT_DICT

{'%22affectation+automatique%22%2Bsite:legifrance.gouv.fr': (['https://www.legifrance.gouv.fr/affichLoiPubliee.do;jsessionid=3349315AB0F643378D8E40169EABCEC8.tpdjo09v_2?idDocument=JORFDOLE000017759359&type=expose&typeLoi=&legislature=12',
   'https://www.legifrance.gouv.fr/download_code_pdf.do?cidTexte=KALITEXT000005670227&pdf=KALITEXT000005670227'],
  4),
 'algorithmes%2Bautomatique+site:%22legifrance.gouv.fr%22': (['https://www.legifrance.gouv.fr/affichCodeArticle.do?cidTexte=LEGITEXT000006072026&idArticle=LEGIARTI000032754948&dateTexte=&categorieLien=cid',
   'https://www.legifrance.gouv.fr/affichJuriJudi.do?oldAction=rechJuriJudi&idTexte=JURITEXT000027596148&fastReqId=468358130',
   'https://www.legifrance.gouv.fr/affichTexteArticle.do;jsessionid=57DC9761CE308AD4BF1BD56E4F808923.tpdila10v_3?idArticle=LEGIARTI000006448126&cidTexte=LEGITEXT000006081162&dateTexte=20150310',
   'https://www.legifrance.gouv.fr/affichJuriJudi.do?idTexte=JURITEXT000033880036',
   'https://www.legifrance.g

# How not to be banned

- Switch user-agent
- Put sleeps between requests


In [44]:
_UA_list = [
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-GB; rv:1.8.1.6) Gecko/20070725 ',
    'Chrome/6.0.472.63 Safari/534.3',
    'Firefox/2.0.0.6',
    'Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.3 (KHTML, like Gecko)',
    'Chrome/6.0.472.63 Safari/534.3',
    'Safari/419.3',
    'Mozilla/5.0',
    'Mozilla/5.0 (Linux; U; Android 0.5; en-us) AppleWebKit/522+ (KHTML, like Gecko) ',
    'Opera/9.00 (Windows NT 5.1; U; en)',
    'Version/3.0 Mobile/1A543a Safari/419.3',
    'Mozilla/5.0 (iPhone; U; CPU like Mac OS X; en) AppleWebKit/420+ (KHTML, like Gecko)',
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:46.0) Gecko/20100101 Firefox/46.0',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:41.0) Gecko/20100101 Firefox/41.0'
]

_QUERIES = [
    'algorithmes%2Bautomatique+site:%22legifrance.gouv.fr%22',
    'classement%2Bautomatique+site:%22legifrance.gouv.fr%22',
    'cryptographie+site:legifrance.gouv.fr',
    'analyse+algorithme+-cryptographie+-cryptographiques+-cryptographique+%2Bsite:legifrance.gouv.fr',
    '%22affectation+automatique%22%2Bsite:legifrance.gouv.fr'
]

RESULT_DICT = {}

In [198]:
def G_free_spider():
    for query in _QUERIES :
        all_links = []
        all_titles = []
        actual_page = 1
        flag_last_page = False
        # Check if the query has already been scrapped.
        if query not in RESULT_DICT.keys():
            while (flag_last_page is False):
                header = _change_identity() # Sleep & change header.
                (link_titles, links, flag_last_page) = _scrap_specific_page(query=query, userAgent=header, page=actual_page)
                all_links += links
                all_titles += link_titles
                actual_page += 1
            # Store results.
            print("All %s pages have been scrapped! (query \"%s\")" % (actual_page, query))
            RESULT_DICT[query] = (all_titles, all_links, actual_page)

        else:
            print("Skipping %s (already been scrapped!)" % query)
            pass


def _scrap_specific_page(query, userAgent, page):
    # Variables
    links = []
    link_titles = []
    flag_last_page = False
    # Request
    google_search = 'https://www.google.fr/search?q='
    header = {'User-Agent': userAgent} # Cause I'm not a bot!
    full_query = google_search + query + '&start=' + str((page - 1) * 10)
    clear_output(wait=True)
    print("Starting the scraping of : %s" % full_query)
    page = request.Request(full_query, headers=header)
    try:
        html = request.urlopen(page)
    except:
        print("HTTPError : sleeping 5s and retrying ...")
        time.sleep(5)
        page = request.Request(full_query, headers=header)
        html = request.urlopen(page)
    # Beautify results
    soup = BeautifulSoup(html, "lxml")
    tables = soup.findAll("h3", {"class": "r"})
    if (len(tables) == 0): # Aucun résultat
        flag_last_page=True
        return (link_titles, links, flag_last_page)
    else:
        flag_last_page = False
        # Get clean links from soup
        for table in tables:
            link = table.find("a", href=True)
            if link['href'].startswith('/url?q=') and 'webcache.googleusercontent.com' not in link['href']:
                link_titles.append(link.text)
                href = link['href'].split('/url?q=')[1].split('&')[0]
                link = parse.unquote(href) # good formating of URL's
                links.append(link)
        return (link_titles, links, flag_last_page)


def _change_identity():
    time.sleep(random.uniform(1, 3)) # Make the spider sleep a random amount of seconds
    random_identity = random.choice(_UA_list)
    
    return random_identity

In [200]:
G_free_spider()

Skipping algorithmes%2Bautomatique+site:%22legifrance.gouv.fr%22 (already been scrapped!)
Skipping classement%2Bautomatique+site:%22legifrance.gouv.fr%22 (already been scrapped!)
Skipping cryptographie+site:legifrance.gouv.fr (already been scrapped!)
Skipping analyse+algorithme+-cryptographie+-cryptographiques+-cryptographique+%2Bsite:legifrance.gouv.fr (already been scrapped!)
Skipping %22affectation+automatique%22%2Bsite:legifrance.gouv.fr (already been scrapped!)


In [201]:
RESULT_DICT

{'%22affectation+automatique%22%2Bsite:legifrance.gouv.fr': (['Convention collective nationale du golf du 13 juillet ... - Légifrance',
   'Convention collective nationale du golf du 13 juillet ... - Légifrance',
   "Accord du 11 décembre 2008 relatif à la mise en place d'un plan d ...",
   'république française - Préfecture de Loire-Atlantique',
   "Cour Administrative d'Appel de Nantes, 4ème chambre, 10/01/2014 ...",
   'Dossiers législatifs - Exposé des motifs - Projet de loi ... - Légifrance',
   'Cour de Cassation, Chambre commerciale, du 8 février 2000, 97 ...',
   "Conseil d'État, 7ème SSJS, 19/06/2015, 376216, Inédit au recueil ...",
   'Cour de Cassation, Chambre commerciale, du 13 mars 1973, 71 ...',
   "Circulaire du 14 avril 1997 relative à l'établissement des listes ...",
   'Dossiers législatifs - Exposé des motifs - LOI n° 2004-1343 du ...',
   'Convention collective nationale du golf du 13 ... - Legifrance'],
  ['http://legifrance.gouv.fr/affichIDCCArticle.do;jsessionid

## SANDBOX

# PICKLE IT

In [202]:
pd.DataFrame.from_dict(RESULT_DICT["algorithmes%2Bautomatique+site:%22legifrance.gouv.fr%22"], orient='index')

AttributeError: 'tuple' object has no attribute 'values'

In [58]:
pd.DataFrame.from_dict(RESULT_DICT["algorithmes%2Bautomatique+site:%22legifrance.gouv.fr%22"][0], orient='columns')

,0
0,https://www.legifrance.gouv.fr/affichCodeArtic...
1,https://www.legifrance.gouv.fr/affichJuriJudi....
2,https://www.legifrance.gouv.fr/affichTexteArti...
3,https://www.legifrance.gouv.fr/affichJuriJudi....
4,https://www.legifrance.gouv.fr/affichTexte.do?...
5,http://circulaire.legifrance.gouv.fr/pdf/2009/...
6,https://www.legifrance.gouv.fr/affichCnil.do?i...
7,https://www.legifrance.gouv.fr/affichTexte.do?...
8,https://www.legifrance.gouv.fr/affichCnil.do;j...
9,https://www.legifrance.gouv.fr/affichJuriJudi....


In [237]:
RESULT_as_df = pd.DataFrame()
df = pd.DataFrame()
for key, value in RESULT_DICT.items():
    df = pd.DataFrame.from_dict(value[0])
    df['query'] = key
    df['link'] = value[1]
    df['n_pages'] = value[2]
    df.rename(columns={0: 'link_title'}, inplace=True)
    RESULT_as_df = pd.concat([RESULT_as_df, df])

RESULT_as_df.reset_index(drop=True, inplace=True)
RESULT_as_df.drop_duplicates(subset=['link'], inplace=True)
# Reorder columns
RESULT_as_df = RESULT_as_df.ix[:, ['query', 'link_title', 'link', 'n_pages']]

In [1]:
RESULT_as_df.head(2)

NameError: name 'RESULT_as_df' is not defined

In [239]:
RESULT_as_df.to_csv('QUERY_RESULTS_google.csv')

In [240]:
folder = open('RESULT_DICT_google','wb') 
pickle.dump(RESULT_DICT, folder)